# Scripts

## Prerequisites
Have `kubectl` and `helm` in `$PATH` or `$HOME/tools` (Grid5k)

In [2]:
import os
path_helm = "$HOME/tools/"
env = os.environ
env["PATH"] = os.environ.get("PATH") + ":" + os.environ.get("HOME") + "/tools"
env["KUBECONFIG"] = os.environ.get("HOME") + "/.kube/config"
# specific Grid5k : add $HOME/tools to PATH as PATH is not set in central Jupyter
#new_path = os.environ.get("PATH") + ":" + os.environ.get("HOME") + "/tools"
#kubeconfig = os.environ.get("HOME") + "/.kube/config"
#env = dict(PATH=new_path, KUBECONFIG=kubeconfig)

shell_log = True

In [2]:
%%capture
%pip install importnb 
%pip install ipywidgets
%pip install tqdm
%pip install termcolor
%pip install prometheus-api-client
%pip install pulp
%pip install graphlib_backport # for Python <3.9

In [ ]:
# Low-level

In [ ]:
import subprocess
import shlex
import datetime
from dateutil.tz import tzlocal
import pandas as pd
import glob
import time
from time import sleep 
def run_command_async(command):
    if (isinstance(command, list) == False):
        commands = [command]
    else:
        commands=command
    processes = [None] * len(commands)
    for i in range(len(commands)):
        processes[i] = subprocess.Popen(shlex.split(commands[i]), stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    return processes

def wait_for_command_async(processes):
    while processes:
        for i, process in enumerate(processes):
            output = process.stdout.readline().decode()
            if output == '' and process.poll() is not None:
                processes.remove(process)
                break
            if output:
                now=datetime.datetime.now(tzlocal())
                strnow = now.strftime("%Y-%m-%d %H:%M:%S")
                print ("Log {0} - {1} : ".format(i,strnow) + output.strip())
    
    rc = process.poll()
    return rc    
def run_command(command, shell=True, log=True):
    print(command)
    if (isinstance(command, list) == False):
        commands = [command]
    else:
        commands=command
    processes = [None] * len(commands)
    for i in range(len(commands)):
        if log == True:
            print (commands[i])
        processes[i] = subprocess.Popen(shlex.split(commands[i]), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=shell, env=env)
    while processes:
        for i, process in enumerate(processes):
            output = process.stdout.readline().decode()
            if output == '' and process.poll() is not None:
                processes.remove(process)
                break
            if output and log:
                now=datetime.datetime.now(tzlocal())
                strnow = now.strftime("%Y-%m-%d %H:%M:%S")
                print ("Log {0} - {1} : ".format(i,strnow) + output.strip())
    
    rc = process.poll()
    return rc
import os
def run_command_os(command):
    os.system(command)
    
class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)
def get_strnow():
    now=datetime.datetime.now(tzlocal())
    return now.strftime("%Y%m%d%H%M%S")



In [ ]:
## Charts

In [3]:
def wait_for_http_status(url, code, duration):
    import requests
    import time
    print
    req_code = None
    start_time = time.time()
    end_time= time.time()
    while req_code != code and (end_time - start_time) < duration :
        try:
            time.sleep(1)
            r = requests.head(url)
            req_code = r.status_code
            print("\rWaiting status {} for {} seconds            ".format(code, end_time - start_time), end="\r", flush=True)
            return 0
        except requests.ConnectionError:
            print("failed to connect")
            return 1
        end_time = time.time()    
       
def install_chart(path, name, params, timeout="120s"):
    with cd(path):
        param_str = ""
        for k in params.keys():
            param_str += "--set {}={} ".format(str(k).replace("!","."), str(params[k]).replace("!","."))
        command = "helm install {} . {} --namespace default --wait --timeout {} ".format(name, param_str, timeout)
        result = run_command(command, log = shell_log, shell=False)
        #result = run_command_os(command)
        return result    
    
def uninstall_chart(name, timeout="120s"):
    result = run_command("helm delete {}".format(name), log = shell_log, shell=False)
    return result

In [ ]:
def get_service_public_address(namespace, node_tier, service_name, port):
    # Get address
    from kubernetes import client, config

    # Configs can be set in Configuration class directly or using helper utility
    config.load_kube_config()
    api_instance = client.CoreV1Api()

    # Listing the cluster nodes
    node_list = api_instance.list_node()
    label_master = "node-role.kubernetes.io/controlplane";
    label_worker = "node-role.kubernetes.io/worker";
    count = 0;
    manager_node = None
    jobmanager_node = None
    taskmanager_nodes = []
    for i, node in enumerate(node_list.items):
        if "tier" in node.metadata.labels and node.metadata.labels["tier"] == node_tier:
            jobmanager_node_name = node.metadata.name
            address = [address.address for address in node.status.addresses if address.type=="InternalIP"][0]
            jobmanager_node = address
    service_list = api_instance.read_namespaced_service(namespace=namespace, name=service_name)
    for port_desc in service_list.spec.ports:
        if port_desc.port==port:
            jobmanager_port = port_desc.node_port
    return ("http://{}:{}".format(jobmanager_node, jobmanager_port), "http://{}:{}".format(jobmanager_node_name, jobmanager_port))

In [ ]:
from kubernetes import client, config
def get_label_nodes(ip_address=False):

        # Configs can be set in Configuration class directly or using helper utility
    config.load_kube_config()
    api_instance = client.CoreV1Api()

    # Listing the cluster nodes
    node_list = api_instance.list_node()
    label_master = "node-role.kubernetes.io/controlplane";
    label_worker = "node-role.kubernetes.io/worker";
    count = 0;
    manager_node = None
    jobmanager_node = None
    taskmanager_nodes = []
    for i, node in enumerate(node_list.items):
        print("%s\t%s" % (node.metadata.name, node.metadata.labels))
        address = [address.address for address in node.status.addresses if address.type=="InternalIP"][0]
        if ("tier" in node.metadata.labels and node.metadata.labels["tier"] == "manager" ):
            
            if ip_address:
                manager_node = address
            else:
                manager_node = node.metadata.name
            

        if ("tier" in node.metadata.labels and node.metadata.labels["tier"] == "jobmanager" ):
            if ip_address:
                jobmanager_node = address
            else:
                jobmanager_node = node.metadata.name

        if ("tier" in node.metadata.labels and node.metadata.labels["tier"] == "taskmanager" ):
            if ip_address:
                taskmanager_nodes = address
            else:            
                taskmanager_nodes.append(node.metadata.name)
    return (manager_node, jobmanager_node, taskmanager_nodes)

In [ ]:
def init_label_nodes(jobmanagers_qty=1, kafka_qty=1):
   
    # Configs can be set in Configuration class directly or using helper utility
    config.load_kube_config()
    api_instance = client.CoreV1Api()

    # Listing the cluster nodes
    node_list = api_instance.list_node()
    label_master = "node-role.kubernetes.io/controlplane";
    label_worker = "node-role.kubernetes.io/worker";
    count = 0;
    manager_node = None
    jobmanager_node = None
    taskmanager_nodes = []
    for i, node in enumerate(node_list.items):
        print("%s\t%s" % (node.metadata.name, node.metadata.labels))
        if (label_worker in node.metadata.labels and node.metadata.labels[label_worker] == "true"):
            if count == 0:
                body = {
                    "metadata": {
                        "labels": {
                            "tier":"manager"
                        }
                    }
                }
                api_response = api_instance.patch_node(node.metadata.name, body)
                manager_node = node.metadata.name
            elif count <= jobmanagers_qty:
                body = {
                    "metadata": {
                        "labels": {
                            "tier":"jobmanager"
                        }
                    }
                }
                api_response = api_instance.patch_node(node.metadata.name, body)
                jobmanager_node = node.metadata.name
            elif count <= jobmanagers_qty + kafka_qty:
                body = {
                    "metadata": {
                        "labels": {
                            "tier":"kafka"
                        }
                    }
                }
                api_response = api_instance.patch_node(node.metadata.name, body)
                jobmanager_node = node.metadata.name                
            else:
                body = {
                    "metadata": {
                        "labels": {
                            "tier":"taskmanager"
                        }
                    }
                }
                api_response = api_instance.patch_node(node.metadata.name, body)
                taskmanager_nodes.append(node.metadata.name)

            count += 1 